In [1]:
!pip install pypdf2
!pip install nltk

In [93]:
from PyPDF2 import PdfReader

def read_pdf(filename) :
  pdf_file = open(filename, 'rb')
  read_pdf = PdfReader(pdf_file)
  content = []
  for i in range(len(read_pdf.pages)):
    content.append(read_pdf.pages[i].extract_text())
  
  return content

In [94]:
def preprocessing(chapter) :
  # 각 챕터들의 페이지 리스트를 input으로 받아 각 페이지에 필요없는 줄 (첫 장은 아래 두 줄, 나머지는 위 아래 각 한 줄을 제거)
  idx_0 = -1
  cnt = 0
  while True :
    if chapter[0][idx_0] == '\n':
      if cnt==1 :
        break
      else :
        cnt += 1
        idx_0 -= 1
    else :
      idx_0 -= 1
  chapter[0] = chapter[0][:idx_0]
  
  for i in range(1, len(chapter)) :
    idx_f, idx_b = 0, -1
    while True :
      if idx_f == len(chapter[i]) :
        break
      if chapter[i][idx_f] != '\n' :
        idx_f += 1
      else :
        idx_f += 1
        break
    while True :
      if idx_b == -len(chapter[i]) :
        break
      if chapter[i][idx_b] != '\n' :
        idx_b -= 1
      else :
        break
    chapter[i] = chapter[i][idx_f:idx_b]

  return chapter

In [167]:
def rmv_enter(texts) :
  result = []
  for text in texts :
    tlist = text.replace('-', '').replace('\n', ' ')
    result.append(tlist)
  return result

In [ ]:
book = read_pdf('Project.pdf')

book

In [232]:
import re

chapter_list = []
chapter = []

q = re.compile('\d{1,2}CHAPTER')

for page in book :
  m = q.search(page)
  if m : 
    chapter_list.append(chapter)
    chapter = []
  chapter.append(page)

chapter_list.append(chapter)

In [233]:
del chapter_list[0] # Delete Table of Contents
len(chapter_list)

8

In [234]:
# from nltk.tokenize import sent_tokenize

# def sentence_tokenize(chapter_full_text):
#     chapter_sentences = []
#     for page_text in chapter_full_text:
#         page_sentences = sent_tokenize(page_text)
#         chapter_sentences.append(page_sentences)

#     return chapter_sentences

In [235]:
for i in range(len(chapter_list)) :
  chapter_list[i] = preprocessing(chapter_list[i])
  chapter_list[i] = rmv_enter(chapter_list[i])
  # chapter_list[i] = remove_problems(chapter_list[i])

  while not (chapter_list[i][0][0]).isupper() :
    chapter_list[i][0] = chapter_list[i][0][1:] 

In [244]:
pprint(chapter_list[0][77])

('As a PhD student at MIT in 1959, I looked around and found that most of my '
 'classmates  were doing research in the area of information theory and coding '
 'theory that had been established by the great researcher, Claude Shannon.  I '
 'judged that he had solved most   of the important problems already. The '
 'research problems that were left were hard and seemed to me to be of lesser '
 'consequence. So I decided to launch out in a new area that no one else had '
 'yet conceived of.  Happily, at MIT I was surrounded by many computers, and '
 'it was clear to me that, sooner or later, these machines would need to '
 'communicate with each other. At the time, there was no effective way for '
 'them to do so and that the solution to this important problem would have '
 'impact.  I had an approach to this problem and so, for my PhD research, I '
 'decided to create a mathematical theory to model, evaluate, design and   '
 'optimize efficient and reliable data networks. What was you

In [314]:
import json

json_data = []
page_num = 1
for ch_idx in range(len(chapter_list)):
    for page_idx in range(len(chapter_list[ch_idx])):
        json_dict = {}
        
        json_dict["id"] = f"Page {page_num}"
        json_dict["page"] = page_num
        json_dict["contents"] = chapter_list[ch_idx][page_idx]
        
        page_num += 1
        json_data.append(json_dict)
    
# json_data

In [299]:
import re

def remove_homework_text(json_data):
  # HW problem에 대응되는 패턴을 찾지 못해 하드코딩...
  exclude_page_range = [
      [67, 77],
      [166, 177],
      [285, 300],
      [365, 374],
      [438, 445],
      [520, 527],
      [598, 602],
      [680, 688]
  ]

  exclude_page_nums = []
  for r in exclude_page_range:
    exclude_page_nums.extend(list(range(r[0], r[1] + 1)))

  filtered = []
  hw_start = "Homework Problems and Questions"
  for page_dict in json_data:
    if page_dict["page"] in exclude_page_nums:
      continue
    else: # 본문과 HW이 연달아 나오는 경우에는 HW만 제외 
      idx = page_dict["contents"].find(hw_start)
      if idx != -1:
        page_dict["contents"] = page_dict["contents"][:idx]

      filtered.append(page_dict)
    
  return filtered

In [315]:
json_data = remove_homework_text(json_data)
json_data[66] # if successful, it's interview page

{'id': 'Page 78',
 'page': 78,
 'contents': 'As a PhD student at MIT in 1959, I looked around and found that most of my classmates  were doing research in the area of information theory and coding theory that had been established by the great researcher, Claude Shannon.  I judged that he had solved most   of the important problems already. The research problems that were left were hard and seemed to me to be of lesser consequence. So I decided to launch out in a new area that no one else had yet conceived of.  Happily, at MIT I was surrounded by many computers, and it was clear to me that, sooner or later, these machines would need to communicate with each other. At the time, there was no effective way for them to do so and that the solution to this important problem would have impact.  I had an approach to this problem and so, for my PhD research, I decided to create a mathematical theory to model, evaluate, design and   optimize efficient and reliable data networks. What was your fir

In [311]:
json_data[0]

{'id': 'Page 1',
 'page': 1,
 'contents': 'Today’s Internet is arguably the largest engineered system ever created by  mankind,  with hundreds of millions of connected computers, communication links, and   switches; with billions of users who connect via laptops, tablets, and smartphones;  and with an array of new Internetconnected “things” including game consoles, surveillance systems, watches, eye glasses, thermostats, and cars. Given that the Inter  net is so large and has so many diverse components and uses, is there any hope of understanding how it works? Are there guiding principles and structure that can provide a foundation for understanding such an amazingly large and complex sys  tem? And if so, is it possible that it actually could be both interesting and fun to  learn about computer networks? Fortunately, the answer to all of these questions is a resounding YES! Indeed, it’s our aim in this book to provide you with a modern introduction to the dynamic field of computer netw

### For BM25 retrieval

In [310]:
with open("./resources/document_page_filtered.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent="\t")

### For Dense passage retrieval

In [1]:
import json

with open("./resources/document_page_filtered.json", "r") as f:
    json_filtered = json.load(f)


In [2]:
import copy

json_dpr = copy.deepcopy(json_filtered)

with open("./resources/document_dpr.json", "w", encoding="utf-8") as f:
    for json_dict in json_dpr:
        del json_dict["page"]
        json_dict["contents"] += "\n"
        json.dump(json_dict, f)
        f.write("\n")
        

### DPR with summarization

In [ ]:
import json

with open("./resources/document_summary.json", "r") as f:
    json_dpr_summary = json.load(f)


In [316]:
import copy

json_dpr = copy.deepcopy(json_dpr_summary)

with open("./resources/document_dpr_summary.json", "w", encoding="utf-8") as f:
    for json_dict in json_dpr:
        del json_dict["page"]
        json_dict["contents"] += "\n"
        json.dump(json_dict, f)
        f.write("\n")
        